# Analyse BlueSeis BSPF Events

With pressure sensor parascientific and new sensor 

In [1]:
import os 
import obspy as obs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# from andbro__savefig import __savefig
from obspy.clients.fdsn import Client

## Configurations

In [2]:
if os.uname().nodename == 'lighthouse':
    root_path = '/home/andbro/'
    data_path = '/home/andbro/kilauea-data/'
    archive_path = '/home/andbro/freenas/'
elif os.uname().nodename == 'kilauea':
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'

In [3]:
config = {}

## first region specified
config['minlatitude'] = 31
config['maxlatitude'] = 35
config['minlongitude'] = -119
config['maxlongitude'] = -114
config['eventfile'] = "BSPF_catalog"

## extended region specified
# config['minlatitude'] = 31 -1
# config['maxlatitude'] = 35 +1
# config['minlongitude'] = -119 -1
# config['maxlongitude'] = -114 +1
# config['eventfile'] = "BSPF_event_catalog_extended"

## specify coordinates of station
config['BSPF_lon'] = np.array([-116.455439])
config['BSPF_lat'] = np.array([33.610643])

config['minmagnitude'] = 2.5

config['tbeg'] = obs.UTCDateTime("2022-10-01")
config['tend'] = obs.UTCDateTime("2023-05-31")


config['trigger'] = data_path+"BSPF/trigger/"

config['outpath'] = data_path+"BSPF/data/"
config['outpath'] = "./"

## adjust eventfile name
config['eventfile'] += f"_{str(config['tbeg'].date).replace('-','')}"
config['eventfile'] += f"_{str(config['tend'].date).replace('-','')}"

In [4]:
from functions.catalog_to_dataframe import __catalog_to_dataframe

In [5]:
def __export_new_events(config, events_old, events):
    
    ## combine new and old catalog
    tmp = pd.concat([__cat_to_df(events_old), __cat_to_df(events)]).reset_index(drop=False)
    ## remove duplicates
    df = tmp.drop_duplicates(subset=['index'], keep=False)
    ## sort and set index
    df = df.sort_index(ascending=False)
    df.set_index('index', inplace=True, drop=True)
    
    ## export new events to pickle file
    print(f" -> export new events: {config['outpath']}new_events.pkl")
    df.to_pickle(config['outpath']+"new_events.pkl")
    
    del tmp, df  

In [6]:
def __add_distances_and_backazimuth(config, df):

    from obspy.geodetics.base import gps2dist_azimuth

    dist = np.zeros(len(df))
    baz = np.zeros(len(df))

    
    for ii, ev in enumerate(df.index):
        try:
            dist[ii], az, baz[ii] = gps2dist_azimuth(config['BSPF_lat'], config['BSPF_lon'],
                                                     df.latitude[ii], df.longitude[ii],
                                                     a=6378137.0, f=0.0033528106647474805
                                                     )
        except:
            print(" -> failed to compute!")
            
    df['backazimuth'] = baz
    df['distances_km'] = dist/1000

    return df

## Get Events

In [7]:
client = Client("USGS")

## events - all in area and time period
events_all = client.get_events(minlatitude=config['minlatitude'], maxlatitude=config['maxlatitude'],
                               minlongitude=config['minlongitude'], maxlongitude=config['maxlongitude'],
                               starttime=config['tbeg'],
                               endtime=config['tend'],
                               )

print(f" -> storing: {config['outpath']}{config['eventfile']}_all.xml")
events_all.write(config['outpath']+config['eventfile']+"_all.xml", format="QUAKEML")

 -> storing: ./BSPF_catalog_20221001_20230531_all.xml


/home/andbro/anaconda3/envs/obs/lib/python3.7/site-packages/obspy/io/quakeml/core.py:1109: UserWarning: 'quakeml:earthquake.usgs.gov/fdsnws/event/1/query?starttime=2022-10-01T00%3A00%3A00.000000&endtime=2023-05-31T00%3A00%3A00.000000&minlatitude=31.0&maxlatitude=35.0&minlongitude=-119.0&maxlongitude=-114.0' is not a valid QuakeML URI. It will be in the final file but note that the file will not be a valid QuakeML file.
  warnings.warn(msg % obj.id)


In [8]:
# ## events smaller than 2.0
# events_1 = client.get_events(minlatitude=config['minlatitude'], maxlatitude=config['maxlatitude'],
#                              minlongitude=config['minlongitude'], maxlongitude=config['maxlongitude'],
#                              starttime=config['tbeg'],
#                              endtime=config['tend'],
#                              maxmagnitude=2.0,
#                             )

# ## events between 2.0 and 3.0 within distance 0.5 degrees
# events_2 = client.get_events(
#                              latitude=config['BSPF_lat'], longitude=config['BSPF_lon'],
#                              starttime=config['tbeg'],
#                              endtime=config['tend'],
#                              minmagnitude=2.0,
#                              maxmagnitude=3.0,
#                              maxradius=0.5,
#                             )

# ## events between 2.0 and 3.0 within distance 2.0 degrees
# events_3 = client.get_events(
#                              latitude=config['BSPF_lat'], longitude=config['BSPF_lon'],
#                              starttime=config['tbeg'],
#                              endtime=config['tend'],
#                              minmagnitude=3.0,
#                              maxmagnitude=5.0,
#                              maxradius=5.0,
#                             )

# ## events larger than 5.0 
# events_4 = events_all.filter("magnitude > 5.0")


# ## join specified event catalogs together
# events = events_2 + events_3 + events_4

# events.plot(projection="local");

# print(f" -> storing: {config['outpath']}{config['eventfile']}_filtered.xml")
# events.write(config['outpath']+config['eventfile']+"_filtered.xml", format="QUAKEML")

### converting to dataframe

In [9]:
## convert catalog object to data frame
# events_df = __catalog_to_dataframe(events)
events_all_df = __catalog_to_dataframe(events_all)

### add distances and backazimuth

In [10]:
## add epicentral distances
# __add_distances_and_backazimuth(config, events_df)
__add_distances_and_backazimuth(config, events_all_df)

latitude   longitude    depth  magnitude type  \
2023-05-30 23:29:26.310  34.112333 -117.497000  16340.0       0.86   ml   
2023-05-30 20:56:30.580  33.443833 -116.644333   6860.0       0.29   ml   
2023-05-30 20:51:53.480  33.440333 -116.434667   7370.0       0.87   ml   
2023-05-30 20:36:58.790  33.864667 -117.503500    550.0       1.00   ml   
2023-05-30 20:18:55.760  32.884833 -116.229333   7150.0       1.33   ml   
...                            ...         ...      ...        ...  ...   
2022-10-01 05:52:50.880  34.192500 -116.824333   6870.0       1.19   ml   
2022-10-01 05:36:11.240  33.489667 -116.507167   9120.0       1.13   ml   
2022-10-01 04:30:50.050  34.657667 -116.335167   3860.0       1.33   ml   
2022-10-01 04:06:52.340  34.236000 -118.594500   2850.0       0.88   ml   
2022-10-01 00:08:34.430  33.991833 -116.690167  15010.0       0.84   ml   

                         backazimuth  distances_km  
2023-05-30 23:29:26.310   119.710058    111.290512  
2023-05-30 20:56:30.580    43.431182     25.499096  
2023-05-30 20:51:53.480   354.173003     18.988008  
2023-05-30 20:36:58.790   105.886367    101.125395  
2023-05-30 20:18:55.760   345.393440     83.210713  
...                              ...           ...  
2022-10-01 05:52:50.880   152.033462     73.003543  
2022-10-01 05:36:11.240    19.684030     14.252022  
2022-10-01 04:30:50.050   185.490185    116.669525  
2022-10-01 04:06:52.340   108.726355    209.598906  
2022-10-01 00:08:34.430   152.728123     47.540840  

[7100 rows x 7 columns]

### store catalogs

In [11]:
## write data frame as pickle file
# events_df.to_pickle(config['outpath']+config['eventfile']+".pkl")
events_all_df.to_pickle(config['outpath']+config['eventfile']+"_all.pkl")

## Request Focal Mechanisms

In [12]:
from functions import request_CMT_events

# !rm ./gcmt_solutions.xml

t1 = {str(config['tbeg'].date).replace('-','')}
t2 = {str(config['tend'].date).replace('-','')}

__request_CMT_events(starttime=config['tbeg'], endtime=config['tend'], outtype="QUAKEML", outfile=config['outpath']+f"gcmt_solutions_{t1}_{t2}.xml")

# cgmt = obs.read_events(config['outpath']+'gcmt_solutions_{t1}_{t2}.xml')

NameError: name '__request_CMT_events' is not defined

## Plot Event Timeline

In [ ]:
events_all_df = events_all_df.rename_axis('timestamp').reset_index()
events_all_df['seconds'] = [abs((events_all_df.timestamp.min()-tt).total_seconds()) for tt in events_all_df.timestamp]

events_df = events_df.rename_axis('timestamp').reset_index()
events_df['seconds'] = [abs((events_df.timestamp.min()-tt).total_seconds()) for tt in events_df.timestamp]
events_df

## Plots

In [ ]:
def __plot_timeline(catalog_df, catalog_all_df):

    fig, ax = plt.subplots(1, 1, figsize=(15,5))

    cax = ax.scatter(catalog_all_df.timestamp, catalog_all_df.distances_km, s=4**catalog_all_df.magnitude, 
                     c=catalog_all_df.magnitude, alpha=0.6, cmap='viridis')

#     ax.scatter(catalog_df.timestamp, catalog_df.distances_km, s=4**catalog_df.magnitude, 
#                facecolors='none', edgecolors='r', lw=1.0, zorder=-1)

    plt.colorbar(cax, ax=ax, pad=0.01)

    ax.set_ylim(bottom=0)
    ax.set_ylabel("Distance (km)", fontsize=14)

    ax.grid(ls=":", zorder=-1)
    
#     ax.set_xlim(events_all_df.timestamp.min().date(), events_all_df.timestamp.max().date())
#     ax.set_xlim(obs.UTCDateTime("2022-09-29"), obs.UTCDateTime("2023-04-03"))

    ax.set_title(f"triggered={len(catalog_df)}")
    
    plt.show();
    return fig

In [ ]:
fig = __plot_timeline(events_df, events_all_df)